In [65]:
!pip install neo4j


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('../03 Info Extraction and Triples/Updated_Data/fta_pop_gdp_cleaned_v2.csv')
df_trades=pd.read_csv('../03 Info Extraction and Triples/Updated_Data/trades_v2.csv')

In [81]:
df_trades['exports_from'].unique()

array(['usa', 'afg', 'alb', 'and', 'ago', 'atg', 'arg', 'arm', 'abw',
       'aus', 'aut', 'aze', 'bhs', 'bhr', 'bgd', 'brb', 'blr', 'bel',
       'blz', 'ben', 'bmu', 'btn', 'bol', 'bih', 'bwa', 'bra', 'iot',
       'brn', 'bgr', 'bfa', 'bdi', 'khm', 'cmr', 'can', 'cpv', 'cym',
       'caf', 'tcd', 'chn', 'cxr', 'cck', 'col', 'com', 'cog', 'cod',
       'cok', 'cri', 'civ', 'hrv', 'cub', 'cyp', 'cuw', 'cze', 'dnk',
       'dji', 'dma', 'dom', 'ecu', 'egy', 'slv', 'gnq', 'eri', 'eth',
       'flk', 'fji', 'fin', 'fra', 'pyf', 'atf', 'gab', 'gmb', 'geo',
       'deu', 'gha', 'gib', 'grd', 'gtm', 'gin', 'gnb', 'guy', 'hti',
       'hnd', 'hkg', 'hun', 'isl', 'ind', 'idn', 'irn', 'irq', 'irl',
       'isr', 'ita', 'jam', 'jpn', 'jor', 'kaz', 'ken', 'kir', 'prk',
       'kor', 'kwt', 'kgz', 'lao', 'lva', 'lbn', 'lso', 'lbr', 'lby',
       'ltu', 'lux', 'mac', 'mkd', 'mdg', 'mwi', 'mys', 'mdv', 'mli',
       'mlt', 'mhl', 'mrt', 'mus', 'myt', 'mex', 'fsm', 'mda', 'mne',
       'msr', 'mar',

In [69]:

def get_query(row):
        str1 = 'MERGE (n1: Country {countryID : $n1}) ON CREATE SET n1.name = $n2,  '
        for num in df.columns[5:-3]:
            if row[num]==row[num]:
                str1+= f'n1.{num}= $n{num}, '
        str1 =str1[:-2]
        return str1

In [70]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
          self.driver.close()

    def trade(self, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_trade, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12)

    @staticmethod
    def load_trade(tx, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):

        query1=(
            "MERGE (p1:Trade { tradeID: $p1 }) ON CREATE SET p1.tradedValue = $p6 "
            "MERGE (p4:Year { year: $p4 })"
            "MERGE (p5:Currency { Currency: $p5 })"
            "MERGE (p8:Product { productID: $p8 }) ON CREATE SET p8.product=$p7 "
            "MERGE (p10:Category { categoryID: $p10}) ON CREATE SET p10.Category = $p9 "
        )

        query2=(
            "MERGE (p12:Section { sectionID: $p12}) ON CREATE SET p12.section = $p11 "
        )

        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query2, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)

    def trade_rel(self, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_trade_rel, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12)

    @staticmethod
    def load_trade_rel(tx, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12):

        query3=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p2:Country { countryID: $p2 }) "
            "MERGE (p1)-[:exportedFrom]->(p2) "
            )

        query4=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p3:Country { countryID: $p3 }) "
            "MERGE (p1)-[:exportedTo]->(p3) "
            )

        query5=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p4:Year { year: $p4 }) "
            "MERGE (p1)-[:tradedYear]->(p4) "
            )

        query6=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p5:Currency { Currency: $p5 }) "
            "MERGE (p1)-[:tradedCurrency]->(p5) "
            )

        query7=(
            "MATCH (p1:Trade { tradeID: $p1 }), (p8:Product { productID: $p8 }) "
            "MERGE (p1)-[:tradedProduct]->(p8) "
            )

        query8=(
            "MATCH (p8:Product { productID: $p8 }), (p10:Category { categoryID: $p10 }) "
            "MERGE (p8)-[:hasCategory]->(p10) "
            )

        query9=(
            "MATCH (p12:Section { sectionID: $p12}), (p10:Category { categoryID: $p10 }) "
            "MERGE (p10)-[:hasSection]->(p12) "
            )

        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query4, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query5, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        # tx.run(query6, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query7, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query8, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        tx.run(query9, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6, p7=p7, p8=p8, p9=p9, p10=p10, p11=p11, p12=p12)
        

    def FTA(self, p1, p2, p3, p4, p5, p6):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(
                self.load_fta, p1, p2, p3, p4, p5, p6)

    @staticmethod
    def load_fta(tx, p1, p2, p3, p4, p5, p6):

        query1 = (
            "MERGE (p1:FTA { ftaID: $p1 }) ON CREATE SET p1.has_fta = $p2, p1.signed_date=$p3, p1.date_inforce=$p4 "
        )

        query2=(
            "MATCH (p1:FTA { ftaID: $p1 }), (p5:Country { countryID: $p5 }) "
            "MERGE (p1)-[:hasFTA]->(p5) "
            )
        query3 = (
            "MATCH (p6:Country { countryID: $p6 }), (p1:FTA { ftaID: $p1 }) MERGE (p1)-[:hasFTA]->(p6) "
        )

        tx.run(query1, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query2, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)
        tx.run(query3, p1=p1, p2=p2, p3=p3, p4=p4, p5=p5, p6=p6)

    def create_country_node(self, row, has_fta):
        with self.driver.session(database="neo4j") as session:
            session.execute_write(self._create_country_node, row, has_fta)

    

    @staticmethod
    def _create_country_node(tx, row, has_fta):
        query=(get_query(row))
        tx.run((query), n1=row['country_code'], n2=row['country'], npopulation=row['population'],
                n4=row['signed_year'], n5=row['inforced_year'],
                ngdp_1960=row['gdp_1960'],  ngdp_1961=row['gdp_1961'],  ngdp_1962=row['gdp_1962'],  ngdp_1963=row['gdp_1963'],  ngdp_1964=row['gdp_1964'],  ngdp_1965=row['gdp_1965'],
                ngdp_1966=row['gdp_1966'],  ngdp_1967=row['gdp_1967'],  ngdp_1968=row['gdp_1968'],  ngdp_1969=row['gdp_1969'],  ngdp_1970=row['gdp_1970'],  ngdp_1971=row['gdp_1971'],
                ngdp_1972=row['gdp_1972'],  ngdp_1973=row['gdp_1973'],  ngdp_1974=row['gdp_1974'],  ngdp_1975=row['gdp_1975'],  ngdp_1976=row['gdp_1976'],  ngdp_1977=row['gdp_1977'], 
                ngdp_1978=row['gdp_1978'],  ngdp_1979=row['gdp_1979'],  ngdp_1980=row['gdp_1980'],  ngdp_1981=row['gdp_1981'],  ngdp_1982=row['gdp_1982'],  ngdp_1983=row['gdp_1983'], 
                ngdp_1984=row['gdp_1984'],  ngdp_1985=row['gdp_1985'],  ngdp_1986=row['gdp_1986'],  ngdp_1987=row['gdp_1987'],  ngdp_1988=row['gdp_1988'],  ngdp_1989=row['gdp_1989'],
                ngdp_1990=row['gdp_1990'],  ngdp_1991=row['gdp_1991'],  ngdp_1992=row['gdp_1992'],  ngdp_1993=row['gdp_1993'],  ngdp_1994=row['gdp_1994'],  ngdp_1995=row['gdp_1995'], 
                ngdp_1996=row['gdp_1996'],  ngdp_1997=row['gdp_1997'],  ngdp_1998=row['gdp_1998'],  ngdp_1999=row['gdp_1999'],  ngdp_2000=row['gdp_2000'],  ngdp_2001=row['gdp_2001'],
                ngdp_2002=row['gdp_2002'],  ngdp_2003=row['gdp_2003'],  ngdp_2004=row['gdp_2004'],  ngdp_2005=row['gdp_2005'],  ngdp_2006=row['gdp_2006'],  ngdp_2007=row['gdp_2007'],
                ngdp_2008=row['gdp_2008'],  ngdp_2009=row['gdp_2009'],  ngdp_2010=row['gdp_2010'],  ngdp_2011=row['gdp_2011'],  ngdp_2012=row['gdp_2012'],  ngdp_2013=row['gdp_2013'],
                ngdp_2014=row['gdp_2014'],  ngdp_2015=row['gdp_2015'],  ngdp_2016=row['gdp_2016'],  ngdp_2017=row['gdp_2017'],  ngdp_2018=row['gdp_2018'],  ngdp_2019=row['gdp_2019'],
                ngdp_2020=row['gdp_2020'],  ngdp_2021=row['gdp_2021']  )


In [71]:
df['signed_date'].fillna(value=pd.to_datetime('12/31/2030'), inplace=True)
df['signed_year'] = pd.DatetimeIndex(df['signed_date']).year
df['date_inforce'].fillna(value=pd.to_datetime('12/31/2030'), inplace=True)
df['inforced_year'] = pd.DatetimeIndex(df['date_inforce']).year
df['country_code']=df['country_code'].str.lower()
df['FTA_id']="fta_"+df['FTA_id'].astype(str)
# df.head(2)

In [72]:
df_trades['trade_id'] = df_trades['trade_id'].str.replace('ported','')
df_trades['trade_id'] = df_trades['trade_id'].str.lower()
# df_trades.head(5)

In [73]:
selected_countries=['are', 'aus', 'bhr', 'can', 'col', 'cri', 'dom', 'gha', 'gtm', 'hnd', 'idn', 'isr', 'kor', 'mar', 'mex', 'mus', 'omn', 'pan', 'per', 'sgp', 'slv', 'tha']
len(selected_countries)

22

In [75]:
# df=df[(df.country_code.isin(['usa','are','gha','tha','mus','idn'])) | (df.has_fta.isin([True])) ]
# shortlisted_country=df[~df.country_code.isin(['usa'])]['country_code']
# print(len(shortlisted_country),list(shortlisted_country))
# exports_year_sl=[ 2015,2016,2017,2018,2019,2020]
# df_trades=df_trades[df_trades.exports_year.isin(exports_year_sl)]
df_trades_imports=df_trades[df_trades.exports_to.isin(list(selected_countries))]
df_trades_exports=df_trades[df_trades.exports_from.isin(list(selected_countries))]
df_trades=pd.concat([df_trades_imports,df_trades_exports])
# df_trades=df_trades[(df_trades.exports_to.isin(list(selected_countries)) | df_trades.exports_from.isin(list(selected_countries)))]

# print(df_trades.shape)
df_trades_imports.exports_to.unique()


array(['aus', 'bhr', 'can', 'col', 'cri', 'dom', 'slv', 'gha', 'gtm',
       'hnd', 'idn', 'isr', 'kor', 'mus', 'mex', 'mar', 'omn', 'pan',
       'per', 'sgp', 'tha', 'are'], dtype=object)

In [79]:
sorted(df_trades_exports.exports_from.unique()),len(df_trades_exports.exports_from.unique())


(['are',
  'aus',
  'bhr',
  'can',
  'col',
  'cri',
  'dom',
  'gha',
  'gtm',
  'hnd',
  'idn',
  'isr',
  'kor',
  'mar',
  'mex',
  'mus',
  'omn',
  'pan',
  'per',
  'slv',
  'tha'],
 21)

In [77]:
df_trades_imports.exports_to.unique(),len(df_trades_imports.exports_to.unique())


(array(['aus', 'bhr', 'can', 'col', 'cri', 'dom', 'slv', 'gha', 'gtm',
        'hnd', 'idn', 'isr', 'kor', 'mus', 'mex', 'mar', 'omn', 'pan',
        'per', 'sgp', 'tha', 'are'], dtype=object),
 22)

In [ ]:
len(df_trades), len(df_trades_exports), len(df_trades_imports)
df_trades.tail()

In [53]:
df_trades['exports_from'].unique(),len(df_trades['exports_from'].unique())


(array(['usa', 'aus', 'bhr', 'can', 'col', 'cri', 'dom', 'slv', 'gtm',
        'hnd', 'idn', 'isr', 'kor', 'mus', 'mar', 'omn', 'pan', 'per',
        'tha', 'are'], dtype=object),
 20)

In [54]:
df_trades['exports_to'].unique(),len(df_trades['exports_to'].unique())

(array(['can', 'col', 'dom', 'slv', 'gha', 'gtm', 'hnd', 'idn', 'isr',
        'kor', 'mus', 'mex', 'mar', 'omn', 'pan', 'per', 'sgp', 'tha',
        'are', 'usa'], dtype=object),
 20)

In [55]:
df_trades['exports_from'].value_counts()

usa    109446
can      7111
kor      6288
tha      5967
aus      5808
isr      5349
idn      5004
col      4766
are      4664
per      4556
cri      4446
gtm      3873
hnd      3577
slv      3188
pan      2713
mar      2611
dom      2570
bhr      1503
mus      1178
omn      1092
Name: exports_from, dtype: int64

In [56]:
if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://50728551.databases.neo4j.io:7687"
    user = "neo4j"
    password = "4qdJFRzdAU9l2qWCCizkH163hvQ9UD9j4rk3sYndmQs"
    app = App(uri, user, password)
    
    # df_trades=df_trades.head(25)

    for idx, row in df.iterrows():
      # if row['country_code']=="usa" or row['country_code']=="afg":  
      #   app.create_country_node(row, False)
      app.create_country_node(row, False)
    
    for idx, row in df.iterrows():
      # if row['country_code']=="usa" or row['country_code']=="afg":  
      #   app.FTA(row['FTA_id'], row['has_fta'], row['signed_year'],row['inforced_year'],row['country_code'],"usa")
      app.FTA(row['FTA_id'], row['has_fta'], row['signed_year'],row['inforced_year'],row['country_code'],"usa")




Failed to write data to connection ResolvedIPv4Address(('34.66.78.163', 7687)) (IPv4Address(('34.66.78.163', 7687)))
Failed to write data to connection IPv4Address(('50728551.databases.neo4j.io', 7687)) (IPv4Address(('34.66.78.163', 7687)))


BufferError: Existing exports of data: object cannot be re-sized

In [ ]:
list(df_trades['exports_from'].unique())[1:]

['aus',
 'bhr',
 'can',
 'col',
 'cri',
 'dom',
 'slv',
 'gtm',
 'hnd',
 'isr',
 'jor',
 'kor',
 'mar',
 'omn',
 'pan',
 'per',
 'tha',
 'are',
 'gbr']

In [ ]:
print('starting exports from')
for country in list(df_trades['exports_from'].unique())[1:]:
    print('working on this country from -->',country)
    a=df_trades[df_trades['exports_from']==country]
    for idx, row in a.iterrows():
        app.trade(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],row['currency'],row['trade_value'],row['item_name'],row['item_id'],row['category_name'],row['category_id'],row['section_name'],row['section_id'])


print('starting exports to')
for country in list(df_trades['exports_to'].unique())[1:]:
    print('working on this country to-->',country)
    a=df_trades[df_trades['exports_to']==country]
    for idx, row in a.iterrows():
        app.trade_rel(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],row['currency'],row['trade_value'],row['item_name'],row['item_id'],row['category_name'],row['category_id'],row['section_name'],row['section_id'])

print('done')
app.close()

starting exports from
working on this country from --> aus


Failed to read from defunct connection IPv4Address(('50728551.databases.neo4j.io', 7687)) (IPv4Address(('34.66.78.163', 7687)))
Transaction failed and will be retried in 0.9523007492875899s (Failed to read from defunct connection IPv4Address(('50728551.databases.neo4j.io', 7687)) (IPv4Address(('34.66.78.163', 7687))))
Failed to read from defunct connection ResolvedIPv4Address(('34.66.78.163', 7687)) (IPv4Address(('34.66.78.163', 7687)))
Unable to retrieve routing information
Transaction failed and will be retried in 1.6421214037525897s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 3.277569731315772s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 9.57234141571237s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 15.892385652328079s (Unable to retrieve routing informa

working on this country from --> bhr
working on this country from --> can
working on this country from --> col
working on this country from --> cri
working on this country from --> dom
working on this country from --> slv
working on this country from --> gtm
working on this country from --> hnd
working on this country from --> isr
working on this country from --> jor
working on this country from --> kor
working on this country from --> mar
working on this country from --> omn
working on this country from --> pan
working on this country from --> per
working on this country from --> tha
working on this country from --> are
working on this country from --> gbr
starting exports to
working on this country to--> chl
working on this country to--> col
working on this country to--> dom


Failed to read from defunct connection IPv4Address(('50728551.databases.neo4j.io', 7687)) (IPv4Address(('34.66.78.163', 7687)))
Transaction failed and will be retried in 1.0936743633733552s (Failed to read from defunct connection IPv4Address(('50728551.databases.neo4j.io', 7687)) (IPv4Address(('34.66.78.163', 7687))))
Failed to read from defunct connection ResolvedIPv4Address(('34.66.78.163', 7687)) (IPv4Address(('34.66.78.163', 7687)))
Unable to retrieve routing information
Transaction failed and will be retried in 1.6805335008722078s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 3.614959528996267s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 6.421469037348485s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 14.27402630744951s (Unable to retrieve routing informa

working on this country to--> slv
working on this country to--> gha
working on this country to--> gtm
working on this country to--> hnd
working on this country to--> isr
working on this country to--> kor
working on this country to--> mex
working on this country to--> mar
working on this country to--> omn
working on this country to--> pan
working on this country to--> per


ClientError: {code: Neo.ClientError.Transaction.TransactionHookFailed} {message: You have exceeded the logical size limit of 400000 relationships in your database (attempt to add 5 relationships would reach 400004 relationships). Please consider upgrading to the next tier.}

In [ ]:

# for idx, row in df_trades.iterrows():
#   app.trade(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],row['currency'],row['trade_value'],row['item_name'],row['item_id'],row['category_name'],row['category_id'],row['section_name'],row['section_id'])


# for idx, row in df_trades.iterrows():
#   app.trade_rel(row['trade_id'], row['exports_from'], row['exports_to'],row['exports_year'],row['currency'],row['trade_value'],row['item_name'],row['item_id'],row['category_name'],row['category_id'],row['section_name'],row['section_id'])

# app.close()